In [1]:
%% Linear norm calculation
% This script is meant to calculate/estimate the operator norm of 
%    L_s^I P_{>} (T - T_s) : h^2 --> h^2
%
% A simplified analytical matrix A_s is constructed by noting the
% dominance of near constant diagonal and some superdiagonal terms of
% A. A small number of elements on top left corner are preserved.

clear;
%% Initialize: adding path (at beginning of script)
added_path = ['../functions']; 
addpath(added_path);

In [2]:
%% Setting parameters
rho = 0.90;
kk = 16; % size of (1,1)-block
ld = 0;  % lower bound of diagonal index
ud = 5;  % uppder bound of diagonal index
n = 256; % size of matrix A

%% for interactive data selection
% fprintf('\n Specify rho:\n')
% rho = input(' rho = ');
% fprintf('\n For construction of a preconditioner, specify:\n')
% kk = input(' K (size of (1,1)-block) = ');
% ld = input(' lower index of diag = ');
% ud = input(' upper index of diag = ');
% trunc = input(' truncate Neumann series at = ');

%% Loading matrix data
% Matrix A calculation for large K (in this script, n) is time
% consuming and so A for different values of rho are calculated and
% stored as binary files.
dir = '../../data/data.matlab/';
beta_opt = 1;
nb2 = n/2;
i_rng = 1:nb2;
sh = 1;
if beta_opt == 0
    beta = 0;
elseif beta_opt == 1
    beta = (1 - sqrt(1-rho^2))/rho;
elseif beta_opt == 2
    beta = (1 - 2*sqrt(1-rho^2))/rho;
end
file = sprintf('amat_b%1d_n%02d_r%04d.bin',...
               beta_opt, log2(n), round(rho*1e4));
filename = strcat(dir, file);
fid = fopen(filename);
A = fread(fid, [n/2, n/2], 'double');
fclose(fid);

In [ ]:


%% Construction of preconditioner

ind_f = 1:kk;                           % indices for finite block
ind_c = kk+1:nb2;                       % complementary indices
nf = length(ind_f);
nc = length(ind_c);

I = eye(nb2);
D = diag( i_rng );
Dinv = 1./D;
L = D - A;

As = zeros(size(A));
Atmp = A(:, ind_c);
sh = -ld+1;
dd = zeros(ud-ld+1,1);
for k = ld:ud                        % As12 and As22: banded, diagonal
    dd(k+sh) = mean(diag(Atmp, -kk+k));
    As = As + dd(k+sh)*diag( ones(nb2-abs(k), 1), k);
end
As(ind_f, ind_f) = triu( A(ind_f, ind_f) );  % As11: upper tri.
As(ind_c, ind_f) = 0;                        % As21: zeros
N = Dinv*As;

Es = I;
for k = 1:trunc
    Es = Es + N^k;
end
Lsinv = Es*Dinv;             % Neumann series expansion

%% 





%% Remove path (at end of script/script clean-up)
rmpath(added_path);
